# Vanilla Encoder-Decoder
This is a basic encoder decoder model with no attention

In [1]:
import numpy as np
import seaborn as sns
import tensorflow as tf

import re 
import os
from datetime import datetime

from nltk.translate.bleu_score import sentence_bleu

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Declare Static Variables

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


These parameters are mostly stolen from the Google Paper, except for embedding dim which is determined from GloVE.

In [4]:
EMBEDDING_DIM = 200
ENCODER_UNITS = 1024
DECODER_UNITS = 1024
BATCH_SIZE = 64

### Load Data

In [5]:
BASE_PATH = '/content/drive/MyDrive/Data/Data'  # on local is path to directory
# BASE_PATH = '../../Data'

FORMAL_PATH_TRAIN = '{}/Supervised Data/Entertainment_Music/S_Formal_EM_Train.txt'.format(BASE_PATH)
INFORMAL_PATH_TRAIN = '{}/Supervised Data/Entertainment_Music/S_Informal_EM_Train.txt'.format(BASE_PATH)

FORMAL_PATH_HOLDOUT = '{}/Supervised Data/Entertainment_Music/S_Formal_EM_ValTest.txt'.format(BASE_PATH)
INFORMAL_PATH_HOLDOUT = '{}/Supervised Data/Entertainment_Music/S_Informal_EM_ValTest.txt'.format(BASE_PATH)

EMBEDDING_PATH = '{}/glove.6B.200d.txt'.format(BASE_PATH)

In [6]:
formal = open(FORMAL_PATH_TRAIN).read()
informal = open(INFORMAL_PATH_TRAIN).read()

formal_holdout = open(FORMAL_PATH_HOLDOUT).read()
informal_holdout = open(INFORMAL_PATH_HOLDOUT).read()

In [7]:
def process_sequence(seq):
    """This inserts a space in between the last word and a period"""
    s = re.sub('([.,!?()])', r' \1 ', seq)
    s = re.sub('\s{2,}', ' ', s)
    
    return '<start> ' + s + ' <end>'

In [8]:
f_corpus = [process_sequence(seq) for seq in formal.split('\n')]
if_corpus = [process_sequence(seq) for seq in informal.split('\n')]

f_holdout = [process_sequence(seq) for seq in formal_holdout.split('\n')]
if_holdout = [process_sequence(seq) for seq in informal_holdout.split('\n')]

### Preprocess data

In [9]:
def tokenize(corpus, tokenizer=None, maxlen=None):
    """ Tokenize data and pad sequences """
    if not tokenizer: 
        tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', 
                              oov_token='<OOV>')
        tokenizer.fit_on_texts(corpus)
    
    seqs = tokenizer.texts_to_sequences(corpus)
    padded_seqs = pad_sequences(seqs, padding='post', maxlen=maxlen)

    return padded_seqs, tokenizer

In [10]:
input_train, input_tokenizer = tokenize(if_corpus)
target_train, target_tokenizer = tokenize(f_corpus)

In [11]:
input_test, _ = tokenize(if_holdout, input_tokenizer, 32)
target_test, _ = tokenize(f_holdout, target_tokenizer)

In [12]:
buffer_size = len(input_train)
steps_per_epoch = len(input_train) // BATCH_SIZE
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

train = tf.data.Dataset.from_tensor_slices((input_train, target_train)).shuffle(buffer_size)
train = train.batch(BATCH_SIZE, drop_remainder=True)

test = tf.data.Dataset.from_tensor_slices((input_test, target_test)).batch(1)

In [13]:
example_input_batch, example_target_batch = next(iter(train))

In [14]:
test_in_batch, test_out_batch = next(iter(test))

# Setup Embedding Weights

In [15]:
def embedding_matrix(tokenizer, vocab_size, embedding_dim):
    embeddings_index = {}
    with open(EMBEDDING_PATH) as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embeddings_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector

    return embeddings_matrix

In [16]:
enc_E_mat = embedding_matrix(input_tokenizer, input_vocab_size, EMBEDDING_DIM)

### Encoder

Using one bidirectional LSTMs because that was reported to get almost as good performance as 2 LSTMs

In [21]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, encoder_units, batch_size,
                 weight_matrix):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.encoder_units = encoder_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, 
                                                   embedding_dim,
                                                   weights=[weight_matrix])
        self.lstm_1 = tf.keras.layers.LSTM(
            self.encoder_units,
            return_sequences=True,
            return_state=True
        )

    def call(self, x, hidden_state):
        """Shove into latent space"""
        # x shape: (batch_size x max_length x embedding_dim)
        output = self.embedding(x)

        # output shape: (batch_size x max_length x 2 * encoder_units)
        # h_f, h_b shapes: (batch_size x encoder_units)
        _, output, _ = self.lstm_1(output)

        return output

In [22]:
encoder = Encoder(input_vocab_size, EMBEDDING_DIM, ENCODER_UNITS, 
                  BATCH_SIZE, enc_E_mat)

init_state = [tf.zeros((BATCH_SIZE, ENCODER_UNITS)) for _ in range(4)]

sample_output = encoder(example_input_batch, init_state)

### Decoder

In [23]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm_1 = tf.keras.layers.LSTM(decoder_units,
                                           return_sequences=True,
                                           return_state=False)
        self.flatten = tf.keras.layers.Flatten()
        self.opt = tf.keras.layers.Dense(vocab_size)

    def call(self, x, encoder_output):
        # x shape: (batch_size, 1, embedding_dim)
        output = self.embedding(x)

        # output shape: (batch_size, 1, decoder_units)
        # this shape is only important for expanding decoder depth
        output = self.lstm_1(output)

        # flatten to feed into opt
        # output shape: (batch_size, hidden_size)
        output = self.flatten(output)

        # get logits
        # x shape: (batch_size, vocab)
        x = self.opt(output)

        return x

In [24]:
decoder = Decoder(target_vocab_size, EMBEDDING_DIM, DECODER_UNITS, BATCH_SIZE)

sample_decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

sample_decoder_output = decoder(sample_decoder_input, sample_output)

### Optimizer and Loss Function

Here we define the optimizer and the loss function. In our loss function we mask the zeros since that's the padding.

Also of note is in the loss function. The reduction argument at default does some really wonky things which threw off all results. Had to change the reduciton to none, which at default is auto. Not exactly sure what it does in this context but it tries to sum over batches. I didn't work with it because I wanted to control all loss calculation manually. 

In [25]:
optimizer = tf.keras.optimizers.Adam()
static_loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [26]:
def loss_function(real, preds):
    """Calculate and return loss"""

    # caclulate loss
    loss = static_loss(real, preds)
    
    # create padding mask 
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss.dtype)
    
    # apply mask
    loss *= mask

    return tf.reduce_mean(loss)

### Training

This is the training loop. I'm using teacher forcing because I don't think I have enough computational power to use beam search. Google reccomends beam search with a beam width of 10, but that isn't an option here. Teacher forcing will provide better results than without using it. 

In [27]:
@tf.function
def train_step(inpt, trgt, init_state):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output = encoder(inpt, init_state)

        # Get start token for every sequence in batch
        dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

        for i in range(1, trgt.shape[1]):
            # dec_hidden shape: (batch_size, decoder_units)
            # dec_input shape: (batch_size, 1)
            predictions = decoder(dec_input, 
                                  enc_output)

            loss += loss_function(trgt[:, i], predictions)
            dec_input = tf.expand_dims(trgt[:, 1], 1)

    # Apply gradients 
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    # return batch loss
    return loss

In [29]:
EPOCHS = 20
# encoder = Encoder(input_vocab_size, EMBEDDING_DIM, ENCODER_UNITS, 
#                   BATCH_SIZE, enc_E_mat)
# decoder = Decoder(target_vocab_size, EMBEDDING_DIM, ATTENTION_UNITS, 
#                   DECODER_UNITS, BATCH_SIZE)

for epoch in range(EPOCHS):
    start = datetime.now()

    total_loss = 0

#     This resets the hidden state of the LSTM for every epoch
#     init_state = [tf.zeros((BATCH_SIZE, ENCODER_UNITS)) for _ in range(4)]
    init_state = None

    for inpt, trgt in train.take(steps_per_epoch):
        total_loss += train_step(inpt, trgt, init_state)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / BATCH_SIZE))
    print('Time taken {}\n'.format(datetime.now() - start))

Epoch 1 Loss 381.7358
Time taken 0:00:45.848956

Epoch 2 Loss 380.7505
Time taken 0:00:45.975878

Epoch 3 Loss 379.9378
Time taken 0:00:45.804792

Epoch 4 Loss 379.1068
Time taken 0:00:45.943130

Epoch 5 Loss 378.4908
Time taken 0:00:45.721870

Epoch 6 Loss 377.8930
Time taken 0:00:45.869635

Epoch 7 Loss 377.4195
Time taken 0:00:45.926568

Epoch 8 Loss 376.9259
Time taken 0:00:45.899789

Epoch 9 Loss 376.5666
Time taken 0:00:45.900474

Epoch 10 Loss 376.2126
Time taken 0:00:45.771482

Epoch 11 Loss 375.8883
Time taken 0:00:45.846654

Epoch 12 Loss 375.5713
Time taken 0:00:45.860442

Epoch 13 Loss 375.2558
Time taken 0:00:45.695137

Epoch 14 Loss 374.9235
Time taken 0:00:45.915196

Epoch 15 Loss 374.7338
Time taken 0:00:45.811437

Epoch 16 Loss 374.4879
Time taken 0:00:45.774587

Epoch 17 Loss 374.3262
Time taken 0:00:45.804848

Epoch 18 Loss 374.1107
Time taken 0:00:45.874961

Epoch 19 Loss 373.9610
Time taken 0:00:45.841723

Epoch 20 Loss 373.8383
Time taken 0:00:45.847088



In [36]:
def evaluate(data):
    loss = 0
    results = []
    bleu = 0

    for i, (x, y) in enumerate(data):

        result = '<start>'
        next_word = True

        init_state = [tf.zeros((1, ENCODER_UNITS)) for _ in range(4)]

        # This feeds in the start token for the first initial state
        dec_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
        
        # Get inputs
        enc_output = encoder(x, init_state)

        j = 1  # iterative count
        while next_word: 
            # dec_hidden shape: (batch_size, decoder_units)
            # dec_input shape: (batch_size, 1)
            predictions = decoder(dec_input, 
                                  enc_output)
            loss += loss_function(y[:, j], predictions)
            
            # max logit for tokenized word
            word_idx = tf.argmax(predictions[0]).numpy()
            word = target_tokenizer.index_word[word_idx]
            result += word + ' '

            dec_input = tf.expand_dims([word_idx], 0)

            if word == '<end>':
                next_word = False
            
            if i >= y.shape[1] - 1:
                result += '<end>'
                next_word = False
            
            j += 1

        results.append(result)
        bleu += sentence_bleu(f_holdout[i], result)

    return results, loss.numpy() / len(f_holdout), bleu / len(f_holdout)

In [37]:
results, test_loss, bleu = evaluate(test)

In [38]:
test_loss

4.785417318711479

In [39]:
bleu

0.9395869798791913

In [40]:
def examine(index):
    print("Informal: ", if_holdout[index])
    print("Formal: ", f_holdout[index])
    print("Predicted: ", results[index])

In [44]:
examine(222)

Informal:  <start> you should give it a try .  <end>
Formal:  <start> You should give it a try .  <end>
Predicted:  <start>i <end>
